In [2]:
import requests
import pandas as pd
import json
import numpy as np

In [7]:
nypd_complaints = pd.read_csv('./crimes-new-york-city/NYPD_Complaint_Data_Historic.csv')

/Users/briansrebrenik/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
nypd_complaints.Lat_Lon.head()

0    (40.828848333, -73.916661142)
1    (40.697338138, -73.784556739)
2    (40.802606608, -73.945051911)
3    (40.654549444, -73.726338791)
4       (40.7380024, -73.98789129)
Name: Lat_Lon, dtype: object

In [9]:
nypd_complaints.columns

Index(['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'RPT_DT', 'KY_CD', 'OFNS_DESC', 'PD_CD', 'PD_DESC',
       'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'JURIS_DESC', 'BORO_NM',
       'ADDR_PCT_CD', 'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'PARKS_NM',
       'HADEVELOPT', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude',
       'Lat_Lon'],
      dtype='object')

In [10]:
nypd_complaints_data = nypd_complaints[['RPT_DT', 'OFNS_DESC', 'PD_DESC', 'Latitude', 'Longitude', 'Lat_Lon']]

In [11]:
nypd_complaints_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
RPT_DT       1048575 non-null object
OFNS_DESC    1045695 non-null object
PD_DESC      1047842 non-null object
Latitude     1016158 non-null float64
Longitude    1016158 non-null float64
Lat_Lon      1016158 non-null object
dtypes: float64(2), object(4)
memory usage: 48.0+ MB


In [12]:
nypd_complaints_data.RPT_DT = nypd_complaints_data.RPT_DT.astype(dtype='datetime64')

/Users/briansrebrenik/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [13]:
nypd_complaints_data_2015 = nypd_complaints_data[nypd_complaints_data['RPT_DT'] > '12-31-2014']

In [14]:
nypd_complaints_data_2015.head()

,RPT_DT,OFNS_DESC,PD_DESC,Latitude,Longitude,Lat_Lon
0,2015-12-31,FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO",40.828848,-73.916661,"(40.828848333, -73.916661142)"
1,2015-12-31,MURDER & NON-NEGL. MANSLAUGHTER,NaN,40.697338,-73.784557,"(40.697338138, -73.784556739)"
2,2015-12-31,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE,INTENT TO",40.802607,-73.945052,"(40.802606608, -73.945051911)"
3,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,40.654549,-73.726339,"(40.654549444, -73.726338791)"
4,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,40.738002,-73.987891,"(40.7380024, -73.98789129)"


In [15]:

nypd_complaints_data_2015.dropna(inplace=True)
nypd_complaints_data_2015.isna().sum()

/Users/briansrebrenik/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


RPT_DT       0
OFNS_DESC    0
PD_DESC      0
Latitude     0
Longitude    0
Lat_Lon      0
dtype: int64

In [16]:
nypd_complaints_data_2015.head()

,RPT_DT,OFNS_DESC,PD_DESC,Latitude,Longitude,Lat_Lon
0,2015-12-31,FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO",40.828848,-73.916661,"(40.828848333, -73.916661142)"
2,2015-12-31,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE,INTENT TO",40.802607,-73.945052,"(40.802606608, -73.945051911)"
3,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,40.654549,-73.726339,"(40.654549444, -73.726338791)"
4,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,40.738002,-73.987891,"(40.7380024, -73.98789129)"
5,2015-12-31,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",40.665023,-73.957111,"(40.665022689, -73.957110763)"


##### getting census tracts

In [5]:
census_data = pd.read_csv('./new-york-city-census-data/census_block_loc.csv')

In [6]:
census_data = census_data[census_data['State'] == "NY"]
census_data.head()

,Latitude,Longitude,BlockCode,County,State
15,40.48,-74.232513,360859901000011,Richmond,NY
16,40.48,-74.229347,360859901000011,Richmond,NY
17,40.48,-74.226181,360859901000011,Richmond,NY
18,40.48,-74.223015,360859901000011,Richmond,NY
19,40.48,-74.219849,360859901000011,Richmond,NY


In [7]:
census_data_nyc_counties = census_data.ix[(census_data['County']=='New York') | (census_data['County']=='Kings') | (census_data['County']=="Richmond") | (census_data['County']=='Bronx')]
def get_closest_block(latitude, longitude):

    temp_df = census_data_nyc_counties.copy()
    temp_df['distance'] = np.sqrt((temp_df.Latitude -latitude)**2 + (temp_df.Longitude - longitude)**2)
    min_distance = min(temp_df.distance)
    return temp_df[temp_df.distance == min_distance].BlockCode
    

    
        #distance = ((x1-x2)**2 + (y1-y2)**2)**0.5
    

/Users/briansrebrenik/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [61]:
list(get_closest_block(nypd_complaints_data_2015.Latitude[10], nypd_complaints_data_2015.Longitude[10]).items())[0][1]

360050119001001

In [8]:
from tqdm import tqdm
def get_tracts(data):
    block_ids = []
    for i in tqdm(range(len(data))):
        try:
            code = list(get_closest_block(data.Latitude[i], data.Longitude[i]).items())[0][1]
            block_ids.append(code)
        except:
            block_ids.append("NA")
    return block_ids

In [67]:
nypd_block_ids = get_tracts(nypd_complaints_data_2015)

100%|██████████| 467109/467109 [43:41<00:00, 178.17it/s]  


In [82]:
nypd_block_ids = pd.DataFrame(nypd_block_ids, columns=['Census Tract'])
nypd_block_ids.head()

,Census Tract
0,360050173001002
1,NA
2,360610198002000
3,360470702031007
4,360610052001002


In [83]:
nypd_block_ids.to_csv('./nypd_block_ids.csv', columns=nypd_block_ids.columns)

In [69]:
len(nypd_block_ids)

467109

In [70]:
nypd_complaints_data_2015['Census Tract'] = nypd_block_ids

/Users/briansrebrenik/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [80]:
nypd_complaints_data_2015= nypd_complaints_data_2015[nypd_complaints_data_2015['Census Tract'] != "NA"]

In [87]:
nypd_complaints_data_2015['Census Tract'] = nypd_complaints_data_2015['Census Tract'].map(lambda x: int(str(x)[:-4]))

In [89]:
nypd_complaints_data_2015.to_csv('./nypd_complaints_data_2015.csv')

#### getting census tract info for 311 data

In [3]:
service_requests = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv')
service_requests.head()

/Users/briansrebrenik/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,32310363,12/31/2015 11:59:45 PM,01/01/2016 12:55:15 AM,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10034.0,71 VERMILYEA AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.865682,-73.923501,"(40.86568153633767, -73.92350095571744)"
1,32309934,12/31/2015 11:59:44 PM,01/01/2016 01:26:57 AM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,11105.0,27-07 23 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.775945,-73.915094,"(40.775945312321085, -73.91509393898605)"
2,32309159,12/31/2015 11:59:29 PM,01/01/2016 04:51:03 AM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,10458.0,2897 VALENTINE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.870325,-73.888525,"(40.870324522111424, -73.88852464418646)"
3,32305098,12/31/2015 11:57:46 PM,01/01/2016 07:43:13 AM,NYPD,New York City Police Department,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,10461.0,2940 BAISLEY AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.835994,-73.828379,"(40.83599404683083, -73.82837939584206)"
4,32306529,12/31/2015 11:56:58 PM,01/01/2016 03:24:42 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11373.0,87-14 57 ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.733060,-73.874170,"(40.733059618956815, -73.87416975810375)"


In [4]:
service_requests_data = service_requests[['Created Date','Complaint Type', 'Descriptor', 'Incident Zip', 'Latitude', 'Longitude']]
service_requests_data.head()

,Created Date,Complaint Type,Descriptor,Incident Zip,Latitude,Longitude
0,12/31/2015 11:59:45 PM,Noise - Street/Sidewalk,Loud Music/Party,10034.0,40.865682,-73.923501
1,12/31/2015 11:59:44 PM,Blocked Driveway,No Access,11105.0,40.775945,-73.915094
2,12/31/2015 11:59:29 PM,Blocked Driveway,No Access,10458.0,40.870325,-73.888525
3,12/31/2015 11:57:46 PM,Illegal Parking,Commercial Overnight Parking,10461.0,40.835994,-73.828379
4,12/31/2015 11:56:58 PM,Illegal Parking,Blocked Sidewalk,11373.0,40.733060,-73.874170


In [9]:
block_ids_311 = get_tracts(service_requests_data)

100%|██████████| 364558/364558 [33:07<00:00, 183.41it/s]


In [10]:
block_ids_311 = pd.DataFrame(block_ids_311, columns=['Census Tract'])
block_ids_311.head()

,Census Tract
0,360610293003000
1,360610240000003
2,360050407023000
3,360050184002005
4,360471180001000


In [11]:
service_requests_data['Census Tract'] = block_ids_311

/Users/briansrebrenik/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
service_requests_data= service_requests_data[service_requests_data['Census Tract'] != "NA"]
service_requests_data['Census Tract'] = service_requests_data['Census Tract'].map(lambda x: int(str(x)[:-4]))

In [ ]:
service_requests_data.to_csv('./service_requests_data.csv')

In [2]:
# AUDREY WORK -- 1/8

In [3]:
nypd = pd.read_csv('./nypd_complaints_data_2015.csv')

In [4]:
nypd.head()

,Unnamed: 0,RPT_DT,OFNS_DESC,PD_DESC,Latitude,Longitude,Lat_Lon,Census Tract
0,0,2015-12-31,FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO",40.828848,-73.916661,"(40.82884833, -73.91666114)",36005017300
1,3,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,40.654549,-73.726339,"(40.65454944, -73.72633879)",36061019800
2,4,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,40.738002,-73.987891,"(40.7380024, -73.98789129)",36047070203
3,5,2015-12-31,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",40.665023,-73.957111,"(40.66502269, -73.95711076)",36061005200
4,6,2015-12-31,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE, POSSESSI",40.720200,-73.988735,"(40.7202, -73.98873508)",36047032500


In [5]:
nypd = nypd.drop(columns=['Latitude', 'Longitude', 'Lat_Lon'])
nypd.head()

,Unnamed: 0,RPT_DT,OFNS_DESC,PD_DESC,Census Tract
0,0,2015-12-31,FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO",36005017300
1,3,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,36061019800
2,4,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,36047070203
3,5,2015-12-31,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",36061005200
4,6,2015-12-31,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE, POSSESSI",36047032500


In [6]:
nypd = nypd.drop(columns=['Unnamed: 0'])
nypd.head()

,RPT_DT,OFNS_DESC,PD_DESC,Census Tract
0,2015-12-31,FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO",36005017300
1,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,36061019800
2,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,36047070203
3,2015-12-31,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",36061005200
4,2015-12-31,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE, POSSESSI",36047032500


In [7]:
nypd.set_index('Census Tract', inplace=True)
nypd.head()

,RPT_DT,OFNS_DESC,PD_DESC
Census Tract,,,
36005017300,2015-12-31,FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO"
36061019800,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3
36047070203,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3
36061005200,2015-12-31,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED"
36047032500,2015-12-31,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE, POSSESSI"


In [8]:
service_reqs = pd.read_csv('service_requests_data.csv')

In [9]:
service_reqs.head()

,Unnamed: 0,Created Date,Complaint Type,Descriptor,Incident Zip,Latitude,Longitude,Census Tracts
0,0,2015-12-31 23:59:45,Noise - Street/Sidewalk,Loud Music/Party,10034.0,40.865682,-73.923501,36061029300
1,1,2015-12-31 23:59:44,Blocked Driveway,No Access,11105.0,40.775945,-73.915094,36061024000
2,2,2015-12-31 23:59:29,Blocked Driveway,No Access,10458.0,40.870325,-73.888525,36005040702
3,3,2015-12-31 23:57:46,Illegal Parking,Commercial Overnight Parking,10461.0,40.835994,-73.828379,36005018400
4,4,2015-12-31 23:56:58,Illegal Parking,Blocked Sidewalk,11373.0,40.733060,-73.874170,36047118000


In [10]:
service_reqs = service_reqs.drop(columns=['Unnamed: 0', 'Latitude', 'Longitude' ])
service_reqs.set_index('Census Tracts', inplace=True)
service_reqs.head()

,Created Date,Complaint Type,Descriptor,Incident Zip
Census Tracts,,,,
36061029300,2015-12-31 23:59:45,Noise - Street/Sidewalk,Loud Music/Party,10034.0
36061024000,2015-12-31 23:59:44,Blocked Driveway,No Access,11105.0
36005040702,2015-12-31 23:59:29,Blocked Driveway,No Access,10458.0
36005018400,2015-12-31 23:57:46,Illegal Parking,Commercial Overnight Parking,10461.0
36047118000,2015-12-31 23:56:58,Illegal Parking,Blocked Sidewalk,11373.0


In [11]:
service_reqs.drop(columns=['Incident Zip'], inplace=True)
service_reqs.head()

,Created Date,Complaint Type,Descriptor
Census Tracts,,,
36061029300,2015-12-31 23:59:45,Noise - Street/Sidewalk,Loud Music/Party
36061024000,2015-12-31 23:59:44,Blocked Driveway,No Access
36005040702,2015-12-31 23:59:29,Blocked Driveway,No Access
36005018400,2015-12-31 23:57:46,Illegal Parking,Commercial Overnight Parking
36047118000,2015-12-31 23:56:58,Illegal Parking,Blocked Sidewalk


In [12]:
service_reqs.columns

Index(['Created Date', 'Complaint Type', 'Descriptor'], dtype='object')

In [13]:
nypd.columns

Index(['RPT_DT', 'OFNS_DESC', 'PD_DESC'], dtype='object')

In [16]:
nypd.rename(columns={'RPT_DT': 'Created Date', 'OFNS_DESC': 'Complaint Type', "PD_DESC": 'Descriptor'}, inplace=True)

In [20]:
nypd = nypd.reset_index()

In [22]:
nypd['311 or NYPD'] = "NYPD"

In [23]:
nypd.head()

,Census Tract,Created Date,Complaint Type,Descriptor,311 or NYPD
0,36005017300,2015-12-31,FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO",NYPD
1,36061019800,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,NYPD
2,36047070203,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,NYPD
3,36061005200,2015-12-31,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",NYPD
4,36047032500,2015-12-31,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE, POSSESSI",NYPD


In [24]:
service_reqs= service_reqs.reset_index()
service_reqs.head()

,Census Tracts,Created Date,Complaint Type,Descriptor
0,36061029300,2015-12-31 23:59:45,Noise - Street/Sidewalk,Loud Music/Party
1,36061024000,2015-12-31 23:59:44,Blocked Driveway,No Access
2,36005040702,2015-12-31 23:59:29,Blocked Driveway,No Access
3,36005018400,2015-12-31 23:57:46,Illegal Parking,Commercial Overnight Parking
4,36047118000,2015-12-31 23:56:58,Illegal Parking,Blocked Sidewalk


In [31]:
service_reqs.rename(columns={'Census Tracts': 'Census Tract'}, inplace=True)

In [25]:
service_reqs['311 or NYPD'] = "311"

In [32]:
service_reqs.head()

,Census Tract,Created Date,Complaint Type,Descriptor,311 or NYPD
0,36061029300,2015-12-31 23:59:45,Noise - Street/Sidewalk,Loud Music/Party,311
1,36061024000,2015-12-31 23:59:44,Blocked Driveway,No Access,311
2,36005040702,2015-12-31 23:59:29,Blocked Driveway,No Access,311
3,36005018400,2015-12-31 23:57:46,Illegal Parking,Commercial Overnight Parking,311
4,36047118000,2015-12-31 23:56:58,Illegal Parking,Blocked Sidewalk,311


In [33]:
all_reports = pd.concat([nypd, service_reqs])

In [34]:
all_reports

,Census Tract,Created Date,Complaint Type,Descriptor,311 or NYPD
0,36005017300,2015-12-31,FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO",NYPD
1,36061019800,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,NYPD
2,36047070203,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,NYPD
3,36061005200,2015-12-31,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",NYPD
4,36047032500,2015-12-31,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE, POSSESSI",NYPD
5,36061001800,2015-12-31,DANGEROUS WEAPONS,WEAPONS POSSESSION 1 & 2,NYPD
6,36005022701,2015-12-31,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,NYPD
7,36005038700,2015-12-31,PETIT LARCENY,"LARCENY,PETIT FROM BUILDING,UN",NYPD
8,36061011800,2015-12-31,PETIT LARCENY,"LARCENY,PETIT OF BICYCLE",NYPD
9,36005011900,2015-12-31,PETIT LARCENY,"LARCENY,PETIT FROM BUILDING,UN",NYPD
